# Twitter Sentiment Analysis

### Step 1: Installing and Setting up Kaggle

In [1]:
# Install the kaggle library
!pip install kaggle

In [2]:
# Create a kaggle directory and copy the kaggle.json file to it
import os
if not os.path.exists(os.path.expanduser("~/.kaggle")):
    os.makedirs(os.path.expanduser("~/.kaggle"))
import shutil
shutil.copy('kaggle.json', os.path.expanduser("~/.kaggle/"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

### Step 2: Importing Twitter Sentiment Dataset

In [3]:
# Download the dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# Extract the compressed dataset
from zipfile import ZipFile
dataset = "sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print("The dataset is extracted")

The dataset is extracted


### Step 3: Importing Dependencies

In [5]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
# Download the stopwords from nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lahar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Step 4: Loading and Preparing the Dataset

In [7]:
# Load the data from the CSV file to a pandas dataframe
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [8]:
# Check the number of rows and columns
print(twitter_data.shape)

(1599999, 6)


In [9]:
# Print the first 5 rows of the dataframe
print(twitter_data.head())

   0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY _TheSpecialOne_  \
0  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   scotthamilton   
1  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY        mattycus   
2  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY         ElleCTF   
3  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          Karoli   
4  0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY        joy_wolf   

  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  
0  is upset that he can't update his Facebook by ...                                                                   
1  @Kenichan I dived many times for the ball. Man...                                                                   
2    my whole body feels itchy and like its on fire                                                                    
3  @nationwideclass no, it's not behaving at all....           

In [10]:
# Rename the columns and read the dataset again
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)

In [11]:
# Check the shape of the dataframe
print(twitter_data.shape)

(1600000, 6)


In [12]:
# Count the number of missing values in the dataset
print(twitter_data.isnull().sum())

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64


In [13]:
# Check the distribution of the target column
print(twitter_data['target'].value_counts())

target
0    800000
4    800000
Name: count, dtype: int64


In [14]:
# Convert the target label "4" to "1"
twitter_data.replace({'target': {4: 1}}, inplace=True)
print(twitter_data['target'].value_counts())

target
0    800000
1    800000
Name: count, dtype: int64


In [15]:
# 0 ---> Negative Tweet
# 1 ---> Positive Tweet

### Step 5: Data Preprocessing and Stemming

In [16]:
# Initialize the Porter Stemmer
port_stem = PorterStemmer()

In [17]:
# Define the stemming function
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
# Apply stemming to the 'text' column
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
# Display the first 5 rows of the dataframe
print(twitter_data.head())

### Step 6: Separating Features and Labels

In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

### Step 7: Splitting the Data into Training and Test Sets

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
# Check the shapes of the datasets
print(X.shape, X_train.shape, X_test.shape)

### Step 8: Converting Textual Data to Numerical Data

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Step 9: Training the Machine Learning Model

In [ ]:
# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

### Step 10: Model Evaluation

In [ ]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data : ', training_data_accuracy)

### Step 11: Saving the Model

In [ ]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Step 12: Using the Model for Future Predictions

In [ ]:
# Load the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
# Evaluate the loaded model on the test data
result = loaded_model.score(X_test, Y_test)
print('Model accuracy on test data: ', result)

In [ ]:
# Make predictions on new data
def make_prediction(index):
    X_new = X_test[index]
    print('Actual label:', Y_test[index])
    prediction = loaded_model.predict(X_new)
    print('Prediction:', prediction)
    if prediction[0] == 0:
        print('Negative tweet')
    else:
        print('Positive tweet')
        
make_prediction(200)
make_prediction(3)